In [1]:
# Cell 1: 安裝必要套件
# 安裝 pandas 用於資料處理
# 安裝 numpy 用於數值運算
# 安裝 requests 用於發送 API 請求
# 安裝 tqdm 用於顯示進度條
# 安裝 pynvml 用於監控 GPU 使用率 (僅適用於 NVIDIA GPU)
# 安裝 matplotlib 用於繪製圖表
# 安裝 python-dotenv 用於載入環境變數
# 安裝 openpyxl 用於讀寫 .xlsx 檔案
!pip install pandas numpy requests tqdm pynvml matplotlib python-dotenv openpyxl

# 程式架構說明:
# 這個程式旨在對評論資料進行多維度分析，並提供人工標註和模型分析的比較功能。
# 程式主要包含以下模組：
# 1. 必要套件安裝 (Cell 1)
# 2. 環境配置與基本設定 (Cell 2)
# 3. 模型 API 適配器定義 (Cell 3) - 處理不同模型的 API 請求
# 4. 檔案和代碼簿相關功能 (Cell 4) - 載入代碼簿和評論資料，構建系統提示訊息
# 5. 模型配置和適配器選擇 (Cell 5) - 獲取模型列表，根據配置選擇適配器
# 6. 評論分析函數 (Cell 6) - 對單條評論進行分析（包含重試邏輯和JSON驗證）
# 7. 批次分析函數 (Cell 7) - 對多條評論進行批次分析，並保存進度
# 8. API 測試函數 (Cell 8) - 測試各類 API 和模型連接
# 9. 人工標註功能 (Cell 9) - 提供人工標註評論的介面，以及比較和統計功能
# 10. 主函數 (Cell 10) - 提供程式的主選單和流程控制
# 11. JSON 資料寫入 SQLite 資料庫 (Cell 13) - 將分析結果保存到資料庫
# 12. 查詢並條列資料庫記錄 (Cell 14) - 顯示資料庫中的記錄
# 13. 清除資料庫記錄 (Cell 15) - 清空資料庫中的分析結果
# 14. 互動模型測試功能 (Cell 12) - 提供一個互動介面來測試單個模型的表現

Defaulting to user installation because normal site-packages is not writeable


In [3]:
pip install ollama

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Cell 2: 引入必要的庫和基本配置
import os # 用於操作檔案和環境變數
import json # 用於處理 JSON 資料
import requests # 用於發送 HTTP 請求
import pandas as pd # 用於資料處理和分析
import numpy as np # 用於數值運算
import time # 用於時間相關操作
import pickle # 用於序列化和反序列化 Python 物件（用於保存進度）
from tqdm import tqdm # 用於顯示進度條
import pynvml # 用於監控 NVIDIA GPU 使用率
import hashlib # 用於生成哈希值 (未使用在此版本)
from datetime import datetime # 用於處理日期和時間
import matplotlib.pyplot as plt # 用於繪製圖表 (未使用在此版本)
from dotenv import load_dotenv # 用於從 .env 檔案載入環境變數

# 在JupyterLab中，在一個新的代碼單元格中運行以下代碼來檢查程式是否有啟動
print("檢查程式是否正常啟動")

# 載入環境變數（用於API密鑰）
load_dotenv() # 從當前目錄或父目錄的 .env 檔案載入變數

# API配置類
class APIConfig:
    """API配置類，儲存各種 API 的端點"""

    # Ollama API 端點
    OLLAMA_API_URL = "http://localhost:11434/api/generate"

    # OpenAI API 端點
    OPENAI_API_URL = "https://api.openai.com/v1/chat/completions"

    # Deepseek API 端點 (如果使用)
    # DEEPSEEK_API_URL = "https://api.deepseek.com/chat/completions"

    # Grok API 端點 (如果使用)
    # GROK_API_URL = "https://api.x.ai/v1/chat/completions"


    # 獲取API密鑰
    @staticmethod
    def get_api_key(provider):
        """根據提供者名稱從環境變數獲取 API 密鑰"""
        key_name = f"{provider.upper()}_API_KEY" # 構建環境變數名稱，例如 OPENAI_API_KEY
        api_key = os.getenv(key_name) # 從環境變數獲取密鑰
        if not api_key:
            # 如果沒有找到密鑰，打印警告信息
            print(f"警告: 未找到 {provider} API密鑰，請在.env檔案中設置 {key_name}")
        return api_key # 返回獲取的 API 密鑰

# 設置檔案路徑
# 代碼簿檔案路徑
codebook_path = 'codebook_v5.xlsx'
# 評論資料檔案路徑
reviews_path = 'hotel_data_20250126_日期與token統計_enriched_20250831_140132_評論數量統計_資料清洗_cl100k_tokens_20250831_145652.xlsx'

# 模型分析結果輸出檔案路徑
output_path = 'multidimensional_analysis_results.json'
# 程式運行日誌檔案路徑
log_path = 'analysis_log.txt'
# 處理進度保存檔案路徑
progress_path = 'processing_progress.pkl'

print("✅ 基本配置完成") # 打印完成信息

檢查程式是否正常啟動
✅ 基本配置完成


In [3]:
# Cell 3: 完整的模型適配器類別定義

# 基類定義
class ModelAPIAdapter:
    """模型API適配器基類，定義了所有適配器應有的基本方法"""

    def __init__(self, model_config):
        """初始化適配器，儲存模型配置"""
        self.model_config = model_config # 儲存傳入的模型配置字典
        self.model_name = model_config.get('name', 'unknown') # 從配置中獲取模型名稱，默認為 'unknown'
        self.model_type = model_config.get('type', 'unknown') # 從配置中獲取模型類型，默認為 'unknown'

    def get_response(self, prompt, review_id=None, log_file=None, query_log_file=None):
        """獲取模型回應的抽象方法（子類必須實現）"""
        # 如果子類沒有實現這個方法，調用時會引發 NotImplementedError
        raise NotImplementedError("子類必須實現此方法")


# Ollama 適配器
class OllamaAdapter(ModelAPIAdapter):
    """Ollama API適配器，用於與本地運行的 Ollama 服務互動"""

    def get_response(self, prompt, review_id=None, log_file=None, query_log_file=None):
        """向Ollama API發送請求並獲取回應"""
        start_time = time.time() # 記錄請求開始時間

        # 記錄開始時間和請求ID到日誌文件（如果提供）
        if log_file:
            log_file.write(f"\n--- 請求開始 ID: {review_id}, 時間: {time.strftime('%Y-%m-%d %H:%M:%S')} ---\n")
            log_file.write(f"模型: {self.model_name}\n")
            log_file.write(f"提示詞長度: {len(prompt)}\n")
            self._log_gpu_usage(log_file) # 記錄 GPU 使用情況

        # 記錄查詢內容到查詢日誌文件（如果提供）
        if query_log_file:
            query_log_file.write(f"\n===== 查詢記錄 =====\n")
            query_log_file.write(f"時間: {time.strftime('%Y-%m-%d %H:%M:%S')}\n")
            query_log_file.write(f"ID: {review_id}\n")
            query_log_file.write(f"模型: {self.model_name}\n")
            query_log_file.write(f"查詢內容:\n{prompt}\n")

        # 準備請求數據，符合 Ollama API 的格式

        data = {
            "model": self.model_name,
            "prompt": prompt,
            "stream": False,
            "format": "json",
            "options": {
                "temperature": 0.4,
                "top_p": 0.85,
                "repeat_penalty": 1.2,
                "num_predict": 1024,
                "num_ctx": 8192,
                "mirostat": 2,
                "mirostat_tau": 5.0,
                "mirostat_eta": 0.1,
                "stop": ["<|end|>"],
                "think": False         # ← 新增：關閉思考模式
            }
        }

        # 發送 API 請求
        try:
            # 使用 requests.post 發送 POST 請求到 Ollama API 端點
            response = requests.post(APIConfig.OLLAMA_API_URL, json=data)
            response.raise_for_status()  # 檢查 HTTP 響應狀態，如果不是 2xx 則引發 HTTPError

            # 從 JSON 響應中提取 'response' 字段作為結果
            result = response.json().get('response', '')

            # 記錄完成時間和 GPU 狀態到日誌文件
            if log_file:
                elapsed = time.time() - start_time # 計算請求耗時
                log_file.write(f"請求完成，耗時: {elapsed:.2f}秒\n")
                log_file.write(f"回應長度: {len(result)}\n")
                self._log_gpu_usage(log_file) # 再次記錄 GPU 使用情況
                log_file.write(f"--- 請求結束 ID: {review_id} ---\n")

            # 記錄回應內容到查詢日誌文件
            if query_log_file:
                query_log_file.write(f"回應內容:\n{result}\n")
                query_log_file.write(f"請求完成，耗時: {time.time() - start_time:.2f}秒\n")
                query_log_file.write(f"===== 記錄結束 =====\n\n")

            return result # 返回模型的回應內容

        except Exception as e:
            # 處理請求過程中發生的錯誤
            error_msg = f"Ollama API請求錯誤: {str(e)}"
            if log_file:
                log_file.write(f"{error_msg}\n") # 將錯誤信息寫入日誌文件

            # 記錄錯誤到查詢日誌文件
            if query_log_file:
                query_log_file.write(f"錯誤: {error_msg}\n")
                query_log_file.write(f"===== 記錄結束 =====\n\n")

            # 重新引發異常以便上層調用者處理
            raise Exception(error_msg)

    def _log_gpu_usage(self, log_file):
        """嘗試記錄 GPU 使用情況（如果 pynvml 可用且是 NVIDIA GPU）"""
        try:
            pynvml.nvmlInit() # 初始化 NVML 庫
            handle = pynvml.nvmlDeviceGetHandleByIndex(0) # 獲取第一個 GPU 的句柄
            mem_info = pynvml.nvmlDeviceGetMemoryInfo(handle) # 獲取顯存信息
            util = pynvml.nvmlDeviceGetUtilizationRates(handle) # 獲取使用率
            # 將 GPU 使用率和顯存信息寫入日誌文件
            log_file.write(f"GPU 使用率: {util.gpu}%, 顯存: {mem_info.used / 1024**2:.2f}/{mem_info.total / 1024**2:.2f} MB\n")
            pynvml.nvmlShutdown() # 關閉 NVML 庫
        except Exception as e:
            # 如果讀取 GPU 信息失敗，記錄錯誤信息
            log_file.write(f"無法讀取GPU資訊: {str(e)}\n")


# OpenAI 適配器
class OpenAIAdapter(ModelAPIAdapter):
    """OpenAI API 適配器，用於與 OpenAI 的模型互動"""

    def __init__(self, model_config):
        """初始化 OpenAI API 適配器，獲取 API 密鑰"""
        super().__init__(model_config) # 調用父類的初始化方法
        # 從模型配置或環境變數獲取 API 密鑰
        self.api_key = model_config.get('api_key') or APIConfig.get_api_key('openai')
        if not self.api_key:
            # 如果沒有找到 API 密鑰，引發 ValueError
            raise ValueError("缺少 OpenAI API 密鑰，請在模型配置或環境變數中設置")

    def get_response(self, prompt, review_id=None, log_file=None, query_log_file=None):
        """向 OpenAI API 發送請求並獲取回應，含退避策略"""
        start_time = time.time() # 記錄請求開始時間
        max_retries = 5 # 最大重試次數
        retry_delay = 1 # 初始重試等待時間（秒）

        # 記錄開始時間和請求ID到日誌文件
        if log_file:
            log_file.write(f"\n--- 請求開始 ID: {review_id}, 時間: {time.strftime('%Y-%m-%d %H:%M:%S')} ---\n")
            log_file.write(f"模型: {self.model_name} (OpenAI API)\n")
            log_file.write(f"提示詞長度: {len(prompt)}\n")

        # 記錄查詢內容到查詢日誌文件
        if query_log_file:
            query_log_file.write(f"\n===== 查詢記錄 =====\n")
            query_log_file.write(f"時間: {time.strftime('%Y-%m-%d %H:%M:%S')}\n")
            query_log_file.write(f"ID: {review_id}\n")
            query_log_file.write(f"模型: {self.model_name}\n")
            query_log_file.write(f"查詢內容:\n{prompt}\n")

        # 準備請求數據 - 使用 OpenAI 格式的 API
        data = {
            "model": self.model_name, # 指定使用的模型名稱
            "messages": [
                {"role": "system", "content": "這是一項研究論文項目，你擔任專業的評論分析助手，請以JSON格式回應。"}, # 系統訊息
                {"role": "user", "content": prompt} # 用戶提示詞
            ],
            "temperature": 0.3, # 設置溫度
            "response_format": {"type": "json_object"} # 要求 JSON 對象格式的回應
        }

        # 準備請求標頭，包含授權信息和內容類型
        headers = {
            "Authorization": f"Bearer {self.api_key}", # API 密鑰用於授權
            "Content-Type": "application/json" # 指定內容類型為 JSON
        }

        # 發送 API 請求 - 含退避策略
        for retry in range(max_retries): # 循環進行重試
            try:
                if retry > 0 and log_file:
                    # 如果是重試，記錄等待信息
                    log_file.write(f"重試 OpenAI API 請求 ({retry}/{max_retries})，等待 {retry_delay} 秒...\n")

                if retry > 0:
                    time.sleep(retry_delay) # 等待一段時間再重試
                    # 指數退避：每次重試都增加等待時間
                    retry_delay *= 2 # 將等待時間加倍

                # 發送 POST 請求到 OpenAI API
                response = requests.post(APIConfig.OPENAI_API_URL, headers=headers, json=data)
                response.raise_for_status()  # 檢查 HTTP 響應狀態

                # 解析回應，從 JSON 響應中提取內容
                response_data = response.json()
                result = response_data.get('choices', [{}])[0].get('message', {}).get('content', '')

                # 記錄完成時間到日誌文件
                if log_file:
                    elapsed = time.time() - start_time # 計算耗時
                    log_file.write(f"請求完成，耗時: {elapsed:.2f}秒\n")
                    log_file.write(f"回應長度: {len(result)}\n")
                    log_file.write(f"--- 請求結束 ID: {review_id} ---\n")

                # 記錄回應內容到查詢日誌文件
                if query_log_file:
                    query_log_file.write(f"回應內容:\n{result}\n")
                    query_log_file.write(f"請求完成，耗時: {time.time() - start_time:.2f}秒\n")
                    query_log_file.write(f"===== 記錄結束 =====\n\n")

                return result # 返回模型的回應內容（通常是 JSON 字符串）

            except requests.exceptions.HTTPError as e:
                # 處理 HTTP 錯誤
                # 特別處理 429 (Too Many Requests) 錯誤
                if e.response.status_code == 429:
                    if log_file:
                        log_file.write(f"OpenAI API 速率限制錯誤 (429)，將重試...\n")
                    if retry == max_retries - 1:  # 如果是最後一次重試仍然失敗
                        error_msg = f"OpenAI API 速率限制錯誤，已重試 {max_retries} 次: {str(e)}"
                        if log_file:
                            log_file.write(f"{error_msg}\n")

                        # 記錄錯誤到查詢日誌文件
                        if query_log_file:
                            query_log_file.write(f"錯誤: {error_msg}\n")
                            query_log_file.write(f"===== 記錄結束 =====\n\n")

                        raise Exception(error_msg) # 引發異常
                    continue  # 繼續重試下一次循環
                else:
                    # 其他 HTTP 錯誤
                    error_msg = f"OpenAI API HTTP 錯誤: {e.response.status_code} - {e.response.text}"
                    if log_file:
                        log_file.write(f"{error_msg}\n")

                    # 記錄錯誤到查詢日誌文件
                    if query_log_file:
                        query_log_file.write(f"錯誤: {error_msg}\n")
                        query_log_file.write(f"===== 記錄結束 =====\n\n")

                    raise Exception(error_msg) # 引發異常

            except Exception as e:
                # 處理其他類型的錯誤
                error_msg = f"OpenAI API 請求錯誤: {str(e)}"
                if log_file:
                    log_file.write(f"{error_msg}\n") # 將錯誤信息寫入日誌文件

                # 記錄錯誤到查詢日誌文件
                if query_log_file:
                    query_log_file.write(f"錯誤: {error_msg}\n")
                    query_log_file.write(f"===== 記錄結束 =====\n\n")

                # 如果所有重試都失敗，最後一次引發異常
                if retry == max_retries - 1:
                     raise Exception(error_msg)

In [4]:
# Cell 4: 檔案和代碼簿相關功能

def check_file_paths():
    """檢查所有必要檔案的路徑，並根據需要創建檔案"""
    # 定義需要檢查的輸入檔案 (是否存在)
    files_to_check = [
        (codebook_path, "代碼簿", False), # 代碼簿檔案，不創建
        (reviews_path, "評論資料", False) # 評論資料檔案，不創建
    ]

    # 定義需要檢查或創建的輸出檔案
    files_to_create = [
        (output_path, "分析結果", True, "[]"), # 分析結果檔案，如果不存在則創建空 JSON 陣列
        (log_path, "日誌檔案", True, ""), # 日誌檔案，如果不存在則創建空檔案
        ('query_log.txt', "查詢日誌檔案", True, "") # 查詢日誌檔案，如果不存在則創建空檔案
    ]

    # 檢查輸入檔案是否存在
    missing_files = []
    for file_path, desc, create in files_to_check:
        if not os.path.exists(file_path):
            missing_files.append(f"{desc} ({file_path})") # 記錄缺失的檔案

    if missing_files:
        # 如果有缺失的輸入檔案，打印錯誤信息並返回 False
        print(f"錯誤: 找不到以下必要檔案:")
        for missing in missing_files:
            print(f"- {missing}")
        print("請確保這些檔案存在於正確位置")
        return False

    # 創建輸出檔案（如果不存在）
    for file_path, desc, create, default_content in files_to_create:
        dir_path = os.path.dirname(file_path) # 獲取檔案所在的目錄
        if dir_path and not os.path.exists(dir_path):
            # 如果目錄不存在，嘗試創建
            try:
                os.makedirs(dir_path)
                print(f"已創建目錄: {dir_path}")
            except Exception as e:
                print(f"無法創建目錄 {dir_path}: {e}")
                return False # 如果創建目錄失敗，返回 False

        if not os.path.exists(file_path) and create:
            # 如果檔案不存在且需要創建，嘗試創建並寫入默認內容
            try:
                with open(file_path, 'w', encoding='utf-8') as f:
                    f.write(default_content)
                print(f"已創建 {desc}: {file_path}")
            except Exception as e:
                print(f"無法創建 {desc} {file_path}: {e}")
                return False # 如果創建檔案失敗，返回 False

    return True # 所有檢查和創建都成功，返回 True

def load_codebook(codebook_path):
    """讀取代碼簿並處理成適合分析的格式"""
    try:
        # 使用 pandas 讀取 Excel 檔案
        codebook_df = pd.read_excel(codebook_path, engine='openpyxl')
        codebook = [] # 初始化一個空列表用於儲存代碼簿維度信息
        # 遍歷 DataFrame 的每一行
        for _, row in codebook_df.iterrows():
            # 提取中文名称（处理两种格式："sentiment" 或 "sentiment (情感)"）
            # 根據括號分割代碼名稱，取最後一部分作為中文名稱
            code_parts = str(row['Code']).split('(')
            name_zh = code_parts[-1].replace(')', '').strip() if len(code_parts) > 1 else code_parts[0]

            # 構建維度字典
            dimension = {
                'name_zh': name_zh, # 中文名稱
                'code': code_parts[0].strip(),  # 取第一部分作为代码
                'criteria': [x.strip() for x in str(row['criteria']).split('/') if x.strip()], # 評估標準，按 '/' 分割並去除空白
                'description': str(row['descriptio']) if pd.notna(row['descriptio']) else "" # 維度說明
            }
            codebook.append(dimension) # 將維度字典添加到列表中
        return codebook # 返回處理好的代碼簿列表
    except Exception as e:
        # 如果載入代碼簿失敗，打印錯誤信息
        print(f"代碼簿加載失敗: {e}")
        # 返回空列表而非引發錯誤，讓程式能繼續執行
        return []

def build_system_message(codebook):
    """根據代碼簿構建模型分析所需的系統提示訊息"""
    # 初始化系統訊息
    system_msg = """這是一項研究論文項目，你擔任專業的評論分析助手，請根據以下維度標準進行分析：\n\n"""

    # 添加各維度說明到系統訊息
    for dim in codebook:
        system_msg += f"**{dim['name_zh']} ({dim['code']})**\n" # 添加維度名稱和代碼
        system_msg += f"評估標準：{', '.join(dim['criteria'])}\n" # 添加評估標準，用逗號分隔
        system_msg += f"說明：{dim['description']}\n\n" # 添加維度說明

    # 添加輸出格式和要求
    system_msg += """\n請嚴格按照以下要求執行：
1. 必須輸出完整的JSON格式，包含所有9個維度
2. 嚴格遵循等級標準，不可使用未定義的等級
3. 每個維度必須包含level、keywords、reasoning三個欄位
4. reasoning必須是繁體中文在50字內
5. 隱藏推論與思考的過程例如<think></think>

範例輸出格式：
{
  "sentiment": {"level": "評估等級", "keywords": ["關鍵詞1", "關鍵詞2"], "reasoning": "分析理由"},
  "sarcasm": {"level": "評估等級", "keywords": ["關鍵詞1"], "reasoning": "分析理由"},
  "comfort": {"level": "評估等級", "keywords": ["關鍵詞1"], "reasoning": "分析理由"},
  "value": {"level": "評估等級", "keywords": ["關鍵詞1"], "reasoning": "分析理由"},
  "facilities": {"level": "評估等級", "keywords": ["關鍵詞1"], "reasoning": "分析理由"},
  "location": {"level": "評估等級", "keywords": ["關鍵詞1"], "reasoning": "分析理由"},
  "staff": {"level": "評估等級", "keywords": ["關鍵詞1"], "reasoning": "分析理由"},
  "cleanliness": {"level": "評估等級", "keywords": ["關鍵詞1"], "reasoning": "分析理由"},
  "food": {"level": "評估等級", "keywords": ["關鍵詞1"], "reasoning": "分析理由"}
}"""
    return system_msg # 返回構建好的系統提示訊息

def load_reviews(reviews_path):
    """讀取評論資料"""
    try:
        # 使用 pandas 讀取 Excel 檔案
        reviews = pd.read_excel(reviews_path, engine='openpyxl')
        print(f"評論資料讀取成功！總筆數: {len(reviews)}")

        # 確保有 'text' 欄位，如果原欄位是 'reviews'，則重命名
        if 'reviews' in reviews.columns:
            reviews = reviews.rename(columns={'reviews': 'text'})

        # 如果沒有 'text' 欄位，嘗試使用第一列作為評論內容
        if 'text' not in reviews.columns:
            if len(reviews.columns) >= 1:
                # 假設第一列為評論內容
                reviews = reviews.rename(columns={reviews.columns[0]: 'text'})
            else:
                # 如果沒有任何列，引發 ValueError
                raise ValueError("無法找到評論內容欄位")

        return reviews # 返回載入的評論 DataFrame
    except Exception as e:
        # 如果讀取評論資料失敗，打印錯誤信息
        print(f"讀取評論資料失敗: {e}")
        # 返回空的 DataFrame
        return pd.DataFrame(columns=['text'])

print("✅ 檔案和代碼簿功能已定義")

✅ 檔案和代碼簿功能已定義


In [5]:
# Cell 5: 模型配置和適配器選擇

def get_model_configs():
    """獲取模型配置列表，包含本地和線上模型"""

    # 從環境變數讀取 API 金鑰
    openai_api_key = os.getenv("OPENAI_API_KEY")
    # deepseek_api_key = os.getenv("DEEPSEEK_API_KEY") # 如果有 Deepseek API
    # grok_api_key = os.getenv("GROK_API_KEY") # 如果有 Grok API

    # 本地模型配置 (使用 Ollama)
    # 列表中的每個字典代表一個模型配置
    local_models = [
        {"name": "deepseek-r1:1.5b", "type": "ollama", "display_name": "DeepSeek R1 1.5B"},
        {"name": "deepseek-r1:8b", "type": "ollama", "display_name": "DeepSeek R1 8B"},
        {"name": "deepseek-r1:14b", "type": "ollama", "display_name": "DeepSeek R1 14B"},
        {"name": "deepseek-r1:32b", "type": "ollama", "display_name": "DeepSeek R1 32B"},
        {"name": "gemma3:1b", "type": "ollama", "display_name": "Gemma 3 1B"},
        {"name": "gemma3:4b", "type": "ollama", "display_name": "Gemma 3 4B"},
        {"name": "gemma3:27b", "type": "ollama", "display_name": "Gemma 3 27B"},
        {"name": "phi4:14b", "type": "ollama", "display_name": "Phi-4 14B"},
        {"name": "phi4-mini:3.8b", "type": "ollama", "display_name": "Phi-4 Mini 3.8B"},
        {"name": "qwen3:0.6b", "type": "ollama", "display_name": "Qwen 3 0.6B"},
        {"name": "qwen3:1.7b", "type": "ollama", "display_name": "Qwen 3 1.7B"},
        {"name": "qwen3:4b", "type": "ollama", "display_name": "Qwen 3 4B"},
        {"name": "qwen3:8b", "type": "ollama", "display_name": "Qwen 3 8B"},
        {"name": "qwen3:14b", "type": "ollama", "display_name": "Qwen 3 14B"},
        {"name": "qwq:32b", "type": "ollama", "display_name": "QWQ 32B"},
        {"name": "granite3.3:2b", "type": "ollama", "display_name": "granite3.3:2b"},
        {"name": "granite3.3:8b", "type": "ollama", "display_name": "granite3.3:8b"},
        {"name": "mistral:7b", "type": "ollama", "display_name": "Mistral 7B"},
        {"name": "mistral-small3.1:24b", "type": "ollama", "display_name": "Mistral Small 3.1 24B"},
        {"name": "llama3.2:1b", "type": "ollama", "display_name": "Llama 3.2 1B"},
        {"name": "llama3.2:3b", "type": "ollama", "display_name": "Llama 3.2 3B"},
        {"name": "gpt-oss:20b", "type": "ollama", "display_name": "gpt-oss:20b"},
        {"name": "cwchang/llama3-taide-lx-8b-chat-alpha1:latest", "type": "ollama", "display_name": "llama3-Taide"},
        {"name": "willqiu/Llama-Breeze2-8B-Instruct:latest", "type": "ollama", "display_name": "Breeze2"},
    ]


    # 線上模型配置 (如果配置了 API 密鑰)
    online_models = []

    # 如果找到 OpenAI API 密鑰，添加 OpenAI 模型配置
    if openai_api_key:
        online_models.append({
            "name": "gpt-4o-mini",
            "type": "openai",
            "display_name": "gpt-4o-mini (OpenAI API)",
            "api_key": openai_api_key # 將密鑰添加到配置中
        })

    # 如果有其他線上 API，類似這樣添加配置
    # if deepseek_api_key:
    #     online_models.append({
    #         "name": "deepseek-chat",
    #         "type": "deepseek",
    #         "display_name": "Deepseek Chat (Deepseek API)",
    #         "api_key": deepseek_api_key
    #     })

    # if grok_api_key:
    #      online_models.append({
    #          "name": "grok-1", # 或其他 Grok 模型名稱
    #          "type": "grok",
    #          "display_name": "Grok 1 (Grok API)",
    #          "api_key": grok_api_key
    #      })


    # 合併本地和線上模型列表並返回
    return local_models + online_models


def get_model_adapter(model_config):
    """根據模型類型獲取適當的適配器實例"""
    model_type = model_config.get('type', '').lower() # 獲取模型類型並轉為小寫

    # 根據模型類型返回對應的適配器實例
    if model_type == 'ollama':
        return OllamaAdapter(model_config)
    elif model_type == 'openai':
        return OpenAIAdapter(model_config)
    # 如果有其他 API 類型，在這裡添加對應的適配器
    # elif model_type == 'deepseek':
    #     return DeepseekAdapter(model_config)
    # elif model_type == 'grok':
    #      return GrokAdapter(model_config)
    else:
        # 如果模型類型未知，引發 ValueError
        raise ValueError(f"未知的模型類型: {model_type}")

def select_reviews_by_token_count(reviews, num_reviews):
    """基於 token 數量選擇代表性評論，以確保樣本具有一定的長度分佈代表性"""
    # 檢查 'token_num' 欄位是否存在，如果不存在則計算
    if 'token_num' not in reviews.columns:
        # 計算每條評論的 token 數量（這裡簡單用詞數代替）
        reviews['token_num'] = reviews['text'].apply(lambda x: len(str(x).split()))

    # 如果 num_reviews 為 None 或大於等於總評論數，則返回所有評論
    if num_reviews is None or num_reviews >= len(reviews):
        return reviews

    # 計算評論 token 數量的平均值
    avg_token = reviews['token_num'].mean()
    # 計算每條評論的 token 數量與平均值的絕對差
    reviews['token_diff'] = abs(reviews['token_num'] - avg_token)
    # 根據 token 差值排序，選擇差值最小（最接近平均值）的 num_reviews 條評論
    selected = reviews.sort_values('token_diff').head(num_reviews)
    # 返回選擇的評論，並移除 'token_diff' 欄位
    return selected.drop(columns=['token_diff'])

print("✅ 模型配置功能已定義")

✅ 模型配置功能已定義


In [6]:
# Cell 6: 評論分析函數

def analyze_review(review, key, system_message, model_config, log_file=None, query_log_file=None):
    """分析單條評論，向模型發送請求並處理回應"""
    MAX_RETRIES = 3 # 定義最大重試次數
    retry_count = 0 # 初始化重試計數器

    while retry_count < MAX_RETRIES: # 在最大重試次數內循環
        try:
            # 處理輸入文字，確保它是字符串，並處理非字符串輸入
            if not isinstance(review, str):
                processed_text = str(review)
            else:
                processed_text = review

            # 限制輸入文字長度，防止過長的提示詞
            processed_text = processed_text[:5000] # 只取前5000個字符

            # 準備提示詞 - 根據重試次數選擇不同的提示詞策略
            if retry_count == 0:
                # 首次嘗試使用完整的系統提示訊息和評論內容
                prompt = f"{system_message}\n\n評論內容：{processed_text}"
            else:
                # 第二次或第三次重試時使用簡化版提示詞，強調 JSON 格式要求
                # 從 codebook 中提取需要的維度格式信息，用於構建簡化提示詞
                dimensions_format = ""
                # 遍歷代碼簿中的每個維度
                for dim in codebook:
                    # 構建評估標準字符串，包含所有標準和 '未提及' 選項
                    criteria_str = '/'.join(dim['criteria'] + ['未提及'])
                    # 為每個維度構建 JSON 格式範例字符串
                    dimensions_format += f'  "{dim["code"]}": {{"level": "{criteria_str}", "keywords": ["關鍵詞1", "關鍵詞2"], "reasoning": "簡要分析理由"}},\n'

                # 移除最後一個維度後多餘的逗號和換行符
                dimensions_format = dimensions_format.rstrip(',\n')

                # 構建簡化提示詞，強調 JSON 格式和維度結構
                simplified_prompt = f"""請嚴格分析以下評論並使用JSON格式輸出結果。不要在JSON前後加任何說明文字。

{{
{dimensions_format}
}}

評論內容：{processed_text}"""
                prompt = simplified_prompt # 使用簡化提示詞

            # 記錄當前使用的提示詞策略到日誌文件
            if log_file:
                if retry_count == 0:
                    log_file.write(f"使用標準提示詞\n")
                else:
                    log_file.write(f"使用簡化提示詞 (重試 {retry_count})\n")

            # 獲取適當的模型適配器實例
            try:
                adapter = get_model_adapter(model_config)
            except Exception as e:
                # 如果獲取適配器失敗，引發 ValueError
                raise ValueError(f"無法創建模型適配器: {str(e)}")

            # 記錄請求資訊到日誌文件
            if log_file:
                log_file.write(f"\n開始分析 ID: {key}, 模型: {model_config.get('display_name', model_config.get('name'))}\n")
                log_file.write(f"評論長度: {len(processed_text)}\n")

            # 向模型發送請求並獲取回應
            response = adapter.get_response(prompt, key, log_file, query_log_file)

            # 解析模型回應，預期為 JSON 格式
            try:
                result = json.loads(response) # 將 JSON 字符串解析為 Python 字典
            except json.JSONDecodeError:
                # 如果 JSON 解析失敗，記錄錯誤並引發 ValueError
                if log_file:
                    log_file.write(f"JSON解析錯誤，原始回應: {response[:200]}...\n") # 記錄回應的前 200 個字符
                raise ValueError("模型回應不是有效的JSON格式")

            # 驗證解析結果是字典類型
            if not isinstance(result, dict):
                raise ValueError(f"模型回應不是字典格式: {type(result)}")

            # 驗證所有代碼簿中定義的維度都存在於模型回應中
            missing_dims = [dim['code'] for dim in codebook if dim['code'] not in result]
            if missing_dims:
                # 如果缺少維度，記錄錯誤並引發 ValueError
                if log_file:
                    log_file.write(f"缺少維度: {missing_dims}\n")
                raise ValueError(f"缺少維度分析結果：{missing_dims}")

            # 驗證每個維度的結構（包含 level, keywords, reasoning 欄位）和等級的有效性
            invalid_dims = [] # 儲存無效維度信息的列表
            # 遍歷代碼簿中的每個維度
            for dim in codebook:
                dim_code = dim['code'] # 獲取維度代碼

                # 檢查結果中是否存在該維度（前面已經檢查過，這裡再次確認）
                if dim_code not in result:
                    invalid_dims.append(f"{dim_code} 不存在")
                    continue

                dim_result = result[dim_code] # 獲取該維度的分析結果

                # 檢查必要欄位 (level, keywords, reasoning) 是否存在
                missing_fields = []
                for field in ['level', 'keywords', 'reasoning']:
                    if field not in dim_result:
                        missing_fields.append(field)

                if missing_fields:
                    invalid_dims.append(f"{dim_code} 缺少欄位: {missing_fields}")
                    continue

                # 檢查等級 (level) 是否在定義的評估標準或 '未提及' 中
                valid_levels = dim['criteria'] + ['未提及']
                if dim_result['level'] not in valid_levels:
                    invalid_dims.append(f"{dim_code} 無效等級: {dim_result['level']} (應為: {', '.join(valid_levels)})")
                    continue

                # 檢查關鍵詞 (keywords) 是否為列表類型
                if not isinstance(dim_result['keywords'], list):
                    invalid_dims.append(f"{dim_code} 關鍵詞非列表: {type(dim_result['keywords'])}")
                    continue

            # 如果有無效維度，記錄錯誤並引發 ValueError
            if invalid_dims:
                if log_file:
                    log_file.write(f"無效維度: {invalid_dims}\n")
                raise ValueError(f"無效維度結構: {invalid_dims}")

            # 如果所有驗證都通過，記錄成功信息到日誌文件
            if log_file:
                log_file.write(f"分析成功, ID: {key}\n")

            return result # 返回有效的分析結果字典

        except Exception as e:
            # 處理分析過程中的錯誤（包括 API 錯誤和驗證錯誤）
            retry_count += 1 # 增加重試計數
            error_msg = f"重試 {retry_count}/{MAX_RETRIES} - 錯誤: {str(e)}"
            if log_file:
                log_file.write(f"{error_msg}\n") # 記錄錯誤信息到日誌文件

            # 記錄錯誤到查詢日誌
            if query_log_file:
                query_log_file.write(f"重試 {retry_count}/{MAX_RETRIES} - 錯誤: {str(e)}\n")

    # 如果所有重試都失敗，返回 None
    return None

print("✅ 評論分析函數已定義")

✅ 評論分析函數已定義


In [7]:
# Cell 7: 批次分析函數

def analyze_reviews(reviews, output_path, log_path, system_message, query_log_path="query_log.txt", num_reviews=None, selected_models=None):
    """批次分析評論，處理多條評論和多個模型"""
    # 選擇待處理評論，可以根據 token 數量進行篩選
    reviews_to_process = select_reviews_by_token_count(reviews, num_reviews)

    # 獲取所有可用的模型配置
    all_models = get_model_configs()

    # 過濾出用戶選定的模型配置
    if selected_models:
        # 根據 selected_models 列表中的模型名稱篩選模型配置
        model_configs = [m for m in all_models if m['name'] in selected_models]
    else:
        # 如果沒有指定 selected_models，則使用所有模型
        model_configs = all_models

    if not model_configs:
        # 如果沒有可用的模型配置，打印錯誤並返回空列表
        print("錯誤: 沒有可用的模型配置")
        return []

    # 處理進度記錄
    try:
        # 嘗試從 progress_path 載入之前保存的進度信息
        if os.path.exists(progress_path):
            with open(progress_path, 'rb') as f:
                progress_info = pickle.load(f)
                print(f"讀取進度: {progress_info['processed']}/{progress_info['total']}")
        else:
            # 如果進度檔案不存在，初始化進度信息
            progress_info = {'processed': 0, 'total': len(reviews_to_process) * len(model_configs)}
    except Exception as e:
        # 如果載入進度失敗，打印錯誤並初始化進度信息
        print(f"無法讀取進度資訊: {e}")
        progress_info = {'processed': 0, 'total': len(reviews_to_process) * len(model_configs)}

    # 讀取現有的分析結果（如果存在）
    existing_data = []
    if os.path.exists(output_path):
        try:
            # 嘗試從 output_path 載入 JSON 數據
            with open(output_path, 'r', encoding='utf-8') as f:
                existing_data = json.load(f)
                print(f"已讀取現有結果: {len(existing_data)}筆")
        except json.JSONDecodeError:
            # 如果檔案存在但不是有效的 JSON，打印警告
            print(f"輸出檔案存在但不是有效的JSON，將創建新檔案")

    # 開始批次分析
    try:
        # 打開日誌檔案和查詢日誌檔案，使用 'a' 模式表示追加寫入
        with open(log_path, 'a', encoding='utf-8') as log_file, open(query_log_path, 'a', encoding='utf-8') as query_log_file:
            # 初始化進度條
            pbar = tqdm(total=progress_info['total'], initial=progress_info['processed'])

            # 遍歷每個模型配置
            for model_config in model_configs:
                model_name = model_config.get('name') # 模型名稱
                model_type = model_config.get('type') # 模型類型
                display_name = model_config.get('display_name', model_name) # 用於顯示的名稱

                log_file.write(f"\n\n===== 開始使用模型: {display_name} ({model_type}) =====\n")
                print(f"\n開始使用模型: {display_name} ({model_type})...")

                # 遍歷每一條評論
                for idx, row in reviews_to_process.iterrows():
                    # 檢查該評論是否已經使用當前模型處理過
                    if any(r.get('model') == model_name and r.get('key') == idx for r in existing_data):
                        pbar.update(1) # 如果已處理，更新進度條並跳過
                        progress_info['processed'] += 1
                        continue

                    # 調用 analyze_review 函數分析單條評論
                    analysis = analyze_review(
                        row['text'], idx, system_message, model_config, log_file, query_log_file
                    )

                    # 構建分析結果記錄
                    metadata = {
                        'model_type': model_type, # 記錄模型類型
                    }

                    # 將評論 DataFrame 中的特定元數據添加到記錄中
                    for k in row.index:
                        if k in ['file_name', 'token_range']: # 可以根據需要添加更多元數據
                            metadata[k] = row[k]

                    record = {
                        "model": model_name, # 模型名稱
                        "key": idx, # 評論的索引或 ID
                        "text": row['text'], # 評論原文
                        "analysis": analysis, # 模型分析結果（字典或 None）
                        "metadata": metadata # 其他元數據
                    }

                    existing_data.append(record) # 將結果添加到列表中

                    # 定期保存結果和進度，避免進程中斷時丟失太多數據
                    # 每處理 5 筆或當處理完最後一個模型和最後一條評論時保存
                    if len(existing_data) % 5 == 0 or (model_config == model_configs[-1] and idx == reviews_to_process.index[-1]):
                        # 保存結果到 JSON 檔案
                        with open(output_path, 'w', encoding='utf-8') as out_file:
                            json.dump(existing_data, out_file, ensure_ascii=False, indent=2)

                        # 保存進度到 Pickle 檔案
                        # progress_info['processed'] += 1 # 進度更新應該在這裡，而不是上面
                        with open(progress_path, 'wb') as f:
                            pickle.dump(progress_info, f)

                    pbar.update(1) # 更新進度條

                    # 對於線上模型 API，為了避免速率限制，添加延遲
                    if model_type == 'openai':
                        time.sleep(2)  # OpenAI API 通常需要較長的間隔
                    elif model_type in ['deepseek', 'grok']:
                        time.sleep(1)  # 其他 API 的間隔（可調整）

            pbar.close() # 關閉進度條

    except KeyboardInterrupt:
        # 如果用戶按下 Ctrl+C 中斷程式
        print("\n處理中斷，保存當前進度...")
        # 保存進度到 Pickle 檔案
        with open(progress_path, 'wb') as f:
            pickle.dump(progress_info, f)
        print(f"進度已保存: {progress_info['processed']}/{progress_info['total']}")

    except Exception as e:
        # 處理批次分析過程中發生的其他錯誤
        print(f"處理時發生錯誤: {e}")
        # 發生錯誤時也保存進度
        with open(progress_path, 'wb') as f:
            pickle.dump(progress_info, f)
        print(f"進度已保存: {progress_info['processed']}/{progress_info['total']}")

    return existing_data # 返回所有分析結果的列表

print("✅ 批次分析函數已定義")

✅ 批次分析函數已定義


In [8]:
# Cell 8: API 測試函數

def test_openai_api():
    """測試 OpenAI API 連接"""
    print("\n開始測試 OpenAI API 連接...")

    # 從環境變數獲取 API 密鑰
    api_key = os.getenv("OPENAI_API_KEY")
    if not api_key:
        print("錯誤: 未找到 OpenAI API 密鑰")
        print("請在 .env 檔案中設置 OPENAI_API_KEY")
        return False # 如果沒有密鑰，返回 False

    print(f"使用 OpenAI API 密鑰 (長度: {len(api_key)})")

    # 準備請求標頭
    headers = {
        "Authorization": f"Bearer {api_key}", # 授權標頭
        "Content-Type": "application/json" # 內容類型
    }

    # 準備測試請求的數據
    data = {
        "model": "gpt-4o-mini", # 使用一個輕量級模型進行測試
        "messages": [
            {"role": "system", "content": "你是一個助手"},
            {"role": "user", "content": "進行連線測試,請回覆-測試成功"}
        ],
        "temperature": 0.3, # 設置溫度
        "max_tokens": 100 # 限制回應的最大 token 數
    }

    # 發送測試請求
    try:
        print("發送測試請求到 OpenAI API...")
        # 發送 POST 請求，設置超時時間為 10 秒
        response = requests.post(APIConfig.OPENAI_API_URL, headers=headers, json=data, timeout=10)
        response.raise_for_status() # 檢查 HTTP 響應狀態

        # 解析回應
        response_data = response.json()
        # 提取模型回應內容
        result = response_data.get('choices', [{}])[0].get('message', {}).get('content', '')

        # 檢查回應內容是否包含預期的字符串
        if "測試成功" in result:
             print(f"測試成功! 回應: {result[:100]}...") # 打印回應前 100 個字符
             return True # 返回 True 表示測試成功
        else:
             print(f"測試失敗! 回應內容不符合預期: {result[:100]}...")
             return False # 返回 False 表示測試失敗


    except requests.exceptions.HTTPError as e:
        # 處理 HTTP 錯誤
        print(f"HTTP 錯誤: {e.response.status_code}")
        print(f"回應內容: {e.response.text}")
        return False # 返回 False

    except Exception as e:
        # 處理其他類型的錯誤
        print(f"OpenAI API 測試錯誤: {str(e)}")
        return False # 返回 False

def test_ollama_api():
    """測試 Ollama API 連接"""
    print("\n開始測試 Ollama API 連接...")

    # 檢查 Ollama 服務是否運行，通過請求 /api/tags 端點獲取已安裝模型列表
    try:
        # 發送 GET 請求到 Ollama 的 /api/tags 端點，設置超時時間為 5 秒
        response = requests.get("http://localhost:11434/api/tags", timeout=5)
        response.raise_for_status() # 檢查 HTTP 響應狀態

        # 解析回應，獲取模型列表
        models = response.json().get('models', [])

        if models:
            # 如果找到模型，表示連接成功
            print(f"Ollama 連接成功! 已安裝 {len(models)} 個模型:")
            for model in models:
                print(f"- {model.get('name')}") # 打印模型名稱
            return True # 返回 True 表示測試成功
        else:
            # 如果沒有找到模型，表示 Ollama 運行中但沒有模型
            print("Ollama 已連接但沒有找到已安裝的模型")
            return False # 返回 False

    except requests.exceptions.ConnectionError:
        # 如果無法連接到 Ollama 服務
        print("錯誤: 無法連接到 Ollama 服務")
        print("請確認 Ollama 服務是否已啟動（默認地址: http://localhost:11434）")
        return False # 返回 False

    except Exception as e:
        # 處理其他類型的錯誤
        print(f"Ollama API 測試錯誤: {str(e)}")
        return False # 返回 False

def test_deepseek_api():
    """測試 Deepseek API 連接"""
    print("\n開始測試 Deepseek API 連接...")

    # 從環境變數獲取 API 密鑰
    api_key = os.getenv("DEEPSEEK_API_KEY")
    if not api_key:
        print("錯誤: 未找到 Deepseek API 密鑰")
        print("請在 .env 檔案中設置 DEEPSEEK_API_KEY")
        return False

    print(f"使用 Deepseek API 密鑰 (長度: {len(api_key)})")

    # 準備請求標頭
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }

    # 準備測試請求的數據
    data = {
        "model": "deepseek-chat", # Deepseek 的聊天模型名稱
        "messages": [
            {"role": "system", "content": "你是一個助手"},
            {"role": "user", "content": "進行連線測試,請回覆-測試成功"}
        ],
        "temperature": 0.3,
        "max_tokens": 100
    }

    # 發送測試請求
    try:
        print("發送測試請求到 Deepseek API...")
        # 注意：這裡使用了 APIConfig.DEEPSEEK_API_URL，請確保該變數已定義
        response = requests.post(APIConfig.DEEPSEEK_API_URL, headers=headers, json=data, timeout=10)
        response.raise_for_status()

        # 解析回應
        response_data = response.json()
        result = response_data.get('choices', [{}])[0].get('message', {}).get('content', '')

        if "測試成功" in result:
            print(f"測試成功! 回應: {result[:100]}...")
            return True
        else:
            print(f"測試失敗! 回應內容不符合預期: {result[:100]}...")
            return False

    except requests.exceptions.HTTPError as e:
        print(f"HTTP 錯誤: {e.response.status_code}")
        print(f"回應內容: {e.response.text}")
        return False

    except Exception as e:
        print(f"Deepseek API 測試錯誤: {str(e)}")
        return False

def test_grok_api():
    """測試 Grok API 連接"""
    print("\n開始測試 Grok API 連接...")

    # 從環境變數獲取 API 密鑰
    api_key = os.getenv("GROK_API_KEY")
    if not api_key:
        print("錯誤: 未找到 Grok API 密鑰")
        print("請在 .env 檔案中設置 GROK_API_KEY")
        return False

    print(f"使用 Grok API 密鑰 (長度: {len(api_key)})")

    # 準備請求標頭
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }

    # 準備測試請求的數據
    data = {
        "model": "grok-3", # Grok 模型名稱
        "messages": [
            {"role": "system", "content": "你是一個助手"},
            {"role": "user", "content": "進行連線測試,請回覆-測試成功"}
        ],
        "temperature": 0.3
    }

    # 發送測試請求
    try:
        print("發送測試請求到 Grok API...")
        # 注意：這裡使用了 Grok 的 API 端點
        response = requests.post("https://api.x.ai/v1/chat/completions", headers=headers, json=data, timeout=10)
        response.raise_for_status()

        # 解析回應
        response_data = response.json()
        result = response_data.get('choices', [{}])[0].get('message', {}).get('content', '')

        if "測試成功" in result:
            print(f"測試成功! 回應: {result[:100]}...")
            return True
        else:
            print(f"測試失敗! 回應內容不符合預期: {result[:100]}...")
            return False

    except requests.exceptions.HTTPError as e:
        print(f"HTTP 錯誤: {e.response.status_code}")
        print(f"回應內容: {e.response.text}")
        return False

    except Exception as e:
        print(f"Grok API 測試錯誤: {str(e)}")
        return False


def test_all_apis():
    """測試所有定義的 API 連接 (Ollama, OpenAI, Deepseek, Grok 等)"""
    print("\n===== 開始測試所有 API 連接 =====")

    results = {} # 儲存測試結果的字典

    # 測試 Ollama API
    results["Ollama"] = test_ollama_api()

    # 測試 OpenAI API (如果配置了密鑰)
    if os.getenv("OPENAI_API_KEY"):
        results["OpenAI"] = test_openai_api()

    # 測試 Deepseek API (如果配置了密鑰)
    if os.getenv("DEEPSEEK_API_KEY"):
        # 注意：需要先在 APIConfig 中定義 DEEPSEEK_API_URL
        # results["Deepseek"] = test_deepseek_api()
        pass # 如果未定義 DEEPSEEK_API_URL，則跳過

    # 測試 Grok API (如果配置了密鑰)
    if os.getenv("GROK_API_KEY"):
        # results["Grok"] = test_grok_api()
        pass # 如果未定義 Grok API URL，則跳過

    # 顯示測試結果摘要
    print("\n===== API 測試結果摘要 =====")
    for api, success in results.items():
        status = "✓ 成功" if success else "✗ 失敗"
        print(f"{api}: {status}")

    # 檢查是否有任何 API 測試成功
    if any(results.values()):
        print("\n至少有一個 API 連接測試成功")
        return True
    else:
        print("\n所有 API 連接測試都失敗")
        print("請檢查您的網路連接和 API 設置")
        return False

def test_all_models():
    """測試所有配置的模型連接，檢查是否能成功獲取簡單回應"""
    print("\n===== 開始測試所有模型連接 =====")

    # 獲取所有模型配置
    all_models = get_model_configs()
    if not all_models:
        print("錯誤: 沒有找到可用的模型配置")
        return False # 如果沒有模型配置，返回 False

    print(f"找到 {len(all_models)} 個模型配置")

    # 測試結果記錄列表
    results = []

    # 測試每個模型
    for model_config in all_models:
        model_name = model_config.get('name') # 模型名稱
        model_type = model_config.get('type') # 模型類型
        display_name = model_config.get('display_name', model_name) # 用於顯示的名稱

        print(f"\n正在測試: {display_name} ({model_type})...")

        # 準備簡單的測試提示詞
        test_prompt = "進行連線測試,請回覆-測試成功"

        try:
            # 嘗試獲取適配器實例
            adapter = get_model_adapter(model_config)

            # 嘗試發送簡單請求並計時
            start_time = time.time()
            # 使用一個假的 review_id "test_id"
            response = adapter.get_response(test_prompt, "test_id")
            elapsed = time.time() - start_time # 計算耗時

            # 檢查回應是否成功
            success = False
            response_preview = response[:50] + "..." if len(response) > 50 else response # 截取回應預覽

            # 簡單檢查回應是否包含成功指示詞或至少有意義的文本
            if "測試成功" in response or "你好" in response or len(response) > 10:
                success = True

            # 記錄測試結果
            result = {
                "model": display_name,
                "type": model_type,
                "success": success,
                "response": response_preview,
                "time": f"{elapsed:.2f}秒"
            }
            results.append(result) # 將結果添加到列表中

            # 顯示測試結果
            status = "✓ 成功" if success else "✗ 失敗"
            print(f"{status} - 回應: {response_preview}")
            print(f"耗時: {elapsed:.2f}秒")

        except Exception as e:
            # 處理測試過程中發生的錯誤
            results.append({
                "model": display_name,
                "type": model_type,
                "success": False,
                "error": str(e), # 記錄錯誤信息
                "time": "N/A"
            })
            print(f"✗ 錯誤: {str(e)}")

    # 顯示總結
    print("\n===== 測試結果摘要 =====")
    success_count = sum(1 for r in results if r.get("success")) # 計算成功數
    print(f"總計模型數: {len(results)}")
    print(f"測試成功: {success_count}")
    print(f"測試失敗: {len(results) - success_count}")

    if success_count == 0:
        print("\n警告: 所有模型測試失敗，請檢查您的網絡連接和API設置")
        return False # 如果所有模型都失敗，返回 False

    # 顯示失敗的模型詳細信息
    if len(results) - success_count > 0:
        print("\n失敗的模型:")
        for result in results:
            if not result.get("success"):
                print(f"- {result['model']} ({result['type']}): " +
                     (f"錯誤: {result.get('error')}" if 'error' in result else f"回應: {result.get('response')}"))

    return success_count > 0 # 如果至少有一個模型測試成功，返回 True

print("✅ API 測試函數已定義")

✅ API 測試函數已定義


In [9]:
# Cell 9: 人工標註功能

def start_human_annotation():
    """啟動人工標註流程"""
    print("\n=== 人工標註模式 ===")

    # 檢查必要檔案
    if not check_file_paths():
        print("檔案檢查失敗，無法進行人工標註")
        return

    # 載入代碼簿
    global codebook
    codebook = load_codebook(codebook_path)
    if not codebook:
        print("錯誤: 無法載入代碼簿")
        return

    # 載入評論資料
    reviews = load_reviews(reviews_path)
    if reviews.empty:
        print("錯誤: 無法載入評論資料")
        return

    print(f"已載入 {len(reviews)} 筆評論")
    print(f"代碼簿包含 {len(codebook)} 個維度")

    # 顯示維度資訊
    print("\n=== 標註維度 ===")
    for i, dim in enumerate(codebook, 1):
        print(f"{i}. {dim['name_zh']} ({dim['code']})")
        print(f"   評估標準: {', '.join(dim['criteria'])}")
        print(f"   說明: {dim['description']}")
        print()

    # 設定輸出檔案
    human_annotation_path = 'human_annotation_results.json'

    # 載入現有標註結果
    existing_annotations = []
    if os.path.exists(human_annotation_path):
        try:
            with open(human_annotation_path, 'r', encoding='utf-8') as f:
                existing_annotations = json.load(f)
            print(f"已載入 {len(existing_annotations)} 筆現有標註")
        except json.JSONDecodeError:
            print("現有標註檔案格式錯誤，將創建新檔案")

    # 確定要標註的評論
    annotated_keys = {ann.get('key') for ann in existing_annotations}
    remaining_reviews = reviews[~reviews.index.isin(annotated_keys)]

    if remaining_reviews.empty:
        print("所有評論都已標註完成！")
        return

    print(f"還有 {len(remaining_reviews)} 筆評論需要標註")

    # 詢問是否開始標註
    start_annotation = input("是否開始人工標註? (y/n): ").lower().strip()
    if start_annotation not in ['y', 'yes', '是']:
        return

    # 詢問標註數量
    try:
        num_to_annotate = input("要標註多少筆評論？(按Enter標註全部): ").strip()
        if num_to_annotate:
            num_to_annotate = int(num_to_annotate)
            remaining_reviews = remaining_reviews.head(num_to_annotate)
        else:
            num_to_annotate = len(remaining_reviews)
    except ValueError:
        print("無效輸入，將標註全部剩餘評論")
        num_to_annotate = len(remaining_reviews)

    print(f"將標註 {len(remaining_reviews)} 筆評論")
    print("\n=== 開始標註 ===")
    print("提示:")
    print("- 按 Ctrl+C 可隨時中斷並保存進度")
    print("- 輸入 'skip' 可跳過當前評論")
    print("- 輸入 'help' 查看維度說明")
    print("- 輸入 'quit' 結束標註")
    print()

    try:
        annotated_count = 0
        for idx, row in remaining_reviews.iterrows():
            print(f"\n{'='*50}")
            print(f"評論 #{idx} ({annotated_count + 1}/{len(remaining_reviews)})")
            print(f"{'='*50}")
            print(f"評論內容:")
            print(f"{row['text']}")
            print(f"{'='*50}")

            # 為每個維度收集標註
            annotation_result = {}

            for dim in codebook:
                while True:
                    print(f"\n【{dim['name_zh']} ({dim['code']})】")
                    print(f"評估標準: {', '.join(dim['criteria'])}")
                    print(f"說明: {dim['description']}")

                    # 顯示選項
                    options = dim['criteria'] + ['未提及']
                    for i, option in enumerate(options, 1):
                        print(f"{i}. {option}")

                    user_input = input(f"請選擇 (1-{len(options)}) 或輸入指令: ").strip()

                    # 處理特殊指令
                    if user_input.lower() == 'skip':
                        print("跳過當前評論")
                        break
                    elif user_input.lower() == 'quit':
                        print("結束標註")
                        raise KeyboardInterrupt
                    elif user_input.lower() == 'help':
                        print("\n=== 維度說明 ===")
                        for help_dim in codebook:
                            print(f"{help_dim['name_zh']}: {help_dim['description']}")
                        print("=" * 30)
                        continue

                    # 處理選項選擇
                    try:
                        choice = int(user_input)
                        if 1 <= choice <= len(options):
                            selected_level = options[choice - 1]

                            # 如果不是"未提及"，詢問關鍵詞
                            keywords = []
                            reasoning = ""

                            if selected_level != '未提及':
                                keywords_input = input("請輸入相關關鍵詞 (用逗號分隔，可留空): ").strip()
                                if keywords_input:
                                    keywords = [kw.strip() for kw in keywords_input.split(',') if kw.strip()]

                                reasoning = input("請輸入分析理由 (可留空): ").strip()
                            else:
                                reasoning = "無相關內容"

                            # 記錄結果
                            annotation_result[dim['code']] = {
                                'level': selected_level,
                                'keywords': keywords,
                                'reasoning': reasoning
                            }

                            print(f"已記錄: {dim['name_zh']} = {selected_level}")
                            break
                        else:
                            print(f"請輸入 1-{len(options)} 之間的數字")
                    except ValueError:
                        print("無效輸入，請輸入數字或指令")

                # 如果用戶選擇跳過，中斷當前評論的標註
                if user_input.lower() == 'skip':
                    break

            # 如果完成了所有維度的標註，保存結果
            if len(annotation_result) == len(codebook):
                record = {
                    'key': idx,
                    'text': row['text'],
                    'annotation': annotation_result,
                    'annotator': 'human',
                    'timestamp': datetime.now().isoformat()
                }

                existing_annotations.append(record)
                annotated_count += 1

                # 每標註5筆就保存一次
                if annotated_count % 5 == 0:
                    with open(human_annotation_path, 'w', encoding='utf-8') as f:
                        json.dump(existing_annotations, f, ensure_ascii=False, indent=2)
                    print(f"\n進度已保存 ({annotated_count}/{len(remaining_reviews)})")

            elif user_input.lower() != 'skip':
                print("標註不完整，該評論未保存")

    except KeyboardInterrupt:
        print(f"\n\n標註中斷，已標註 {annotated_count} 筆評論")

    # 最終保存
    with open(human_annotation_path, 'w', encoding='utf-8') as f:
        json.dump(existing_annotations, f, ensure_ascii=False, indent=2)

    print(f"\n=== 標註完成 ===")
    print(f"總計標註: {annotated_count} 筆")
    print(f"結果已保存至: {human_annotation_path}")


def export_annotation_comparison():
    """匯出人工標註與模型分析的比較結果"""
    print("\n=== 匯出標註比較 ===")

    # 檢查檔案
    human_annotation_path = 'human_annotation_results.json'
    model_results_path = 'multidimensional_analysis_results.json'

    if not os.path.exists(human_annotation_path):
        print("錯誤: 找不到人工標註結果檔案")
        return

    if not os.path.exists(model_results_path):
        print("錯誤: 找不到模型分析結果檔案")
        return

    # 載入資料
    with open(human_annotation_path, 'r', encoding='utf-8') as f:
        human_results = json.load(f)

    with open(model_results_path, 'r', encoding='utf-8') as f:
        model_results = json.load(f)

    print(f"人工標註: {len(human_results)} 筆")
    print(f"模型分析: {len(model_results)} 筆")

    # 建立比較資料
    comparison_data = []

    for human_record in human_results:
        human_key = human_record['key']
        human_annotation = human_record['annotation']

        # 找到對應的模型分析結果
        matching_model_records = [r for r in model_results if r.get('key') == human_key]

        for model_record in matching_model_records:
            model_name = model_record.get('model', 'unknown')
            model_analysis = model_record.get('analysis', {})

            # 比較每個維度
            for dim_code in human_annotation.keys():
                if dim_code in model_analysis:
                    human_level = human_annotation[dim_code].get('level', '')
                    model_level = model_analysis[dim_code].get('level', '')

                    comparison_data.append({
                        'review_key': human_key,
                        'model': model_name,
                        'dimension': dim_code,
                        'human_level': human_level,
                        'model_level': model_level,
                        'match': human_level == model_level,
                        'human_keywords': ', '.join(human_annotation[dim_code].get('keywords', [])),
                        'model_keywords': ', '.join(model_analysis[dim_code].get('keywords', [])),
                        'human_reasoning': human_annotation[dim_code].get('reasoning', ''),
                        'model_reasoning': model_analysis[dim_code].get('reasoning', '')
                    })

    # 轉換為DataFrame並匯出
    if comparison_data:
        df = pd.DataFrame(comparison_data)

        # 計算準確率
        accuracy_by_model = df.groupby('model')['match'].mean()
        accuracy_by_dimension = df.groupby('dimension')['match'].mean()

        print(f"\n=== 準確率統計 ===")
        print("各模型準確率:")
        for model, acc in accuracy_by_model.items():
            print(f"  {model}: {acc:.3f}")

        print("\n各維度準確率:")
        for dim, acc in accuracy_by_dimension.items():
            print(f"  {dim}: {acc:.3f}")

        # 匯出詳細比較
        output_file = 'annotation_comparison.xlsx'
        with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
            df.to_excel(writer, sheet_name='詳細比較', index=False)
            accuracy_by_model.to_excel(writer, sheet_name='模型準確率')
            accuracy_by_dimension.to_excel(writer, sheet_name='維度準確率')

        print(f"\n比較結果已匯出至: {output_file}")
    else:
        print("沒有找到可比較的資料")


def view_annotation_stats():
    """查看標註統計資訊"""
    print("\n=== 標註統計 ===")

    human_annotation_path = 'human_annotation_results.json'
    if not os.path.exists(human_annotation_path):
        print("錯誤: 找不到人工標註結果檔案")
        return

    with open(human_annotation_path, 'r', encoding='utf-8') as f:
        human_results = json.load(f)

    if not human_results:
        print("沒有標註資料")
        return

    print(f"總標註筆數: {len(human_results)}")

    # 統計各維度的標註分布
    global codebook
    if not codebook:
        codebook = load_codebook(codebook_path)

    for dim in codebook:
        dim_code = dim['code']
        dim_name = dim['name_zh']

        # 收集該維度的所有標註
        levels = []
        for record in human_results:
            if dim_code in record.get('annotation', {}):
                level = record['annotation'][dim_code].get('level', '')
                levels.append(level)

        if levels:
            print(f"\n{dim_name} ({dim_code}):")
            level_counts = pd.Series(levels).value_counts()
            for level, count in level_counts.items():
                percentage = count / len(levels) * 100
                print(f"  {level}: {count} ({percentage:.1f}%)")

print("✅ 人工標註功能已定義")

✅ 人工標註功能已定義


In [12]:
# Cell 10: 主函數

def main():
    """主程序入口點"""
    # 顯示主選單
    print("\n=== 評論多維度分析工具 ===")
    print("1. 進行評論分析")
    print("2. 測試 API 連接")
    print("3. 測試所有 LLM 模型")
    print("4. 人工標註相關功能")
    print("5. 互動模型測試")  # 新增選項
    print("0. 退出程式")

    try:
        choice = int(input("請選擇操作: "))

        if choice == 0:
            print("程式已退出")
            return
        elif choice == 5:  # 新增選項處理
            start_interactive_testing()

            # 詢問是否返回主選單
            if input("\n是否返回主選單? (y/n): ").lower().strip() in ['y', 'yes', '是']:
                main()
            return
        elif choice == 4:
            # 顯示人工標註子選單
            print("\n=== 人工標註選單 ===")
            print("1. 開始人工標註")
            print("2. 查看標註統計")
            print("3. 匯出標註比較")
            print("0. 返回主選單")

            try:
                annotation_choice = int(input("請選擇操作: "))

                if annotation_choice == 0:
                    main()  # 返回主選單
                    return
                elif annotation_choice == 1:
                    start_human_annotation()
                elif annotation_choice == 2:
                    view_annotation_stats()
                elif annotation_choice == 3:
                    export_annotation_comparison()
                else:
                    print("請選擇有效的選項")
            except ValueError:
                print("請輸入有效的數字")

            # 詢問是否返回主選單
            if input("\n是否返回主選單? (y/n): ").lower().strip() in ['y', 'yes', '是']:
                main()
            return

        elif choice == 2:
            # 顯示 API 測試子選單
            print("\n=== API 測試選單 ===")
            print("1. 測試所有 API 連接")
            print("2. 測試 Ollama API")
            print("3. 測試 OpenAI API")
            print("4. 測試 Deepseek API")
            print("5. 測試 Grok API")
            print("0. 返回主選單")

            try:
                api_choice = int(input("請選擇要測試的 API: "))

                if api_choice == 0:
                    main()  # 返回主選單
                    return
                elif api_choice == 1:
                    test_all_apis()
                elif api_choice == 2:
                    test_ollama_api()
                elif api_choice == 3:
                    test_openai_api()
                elif api_choice == 4:
                    test_deepseek_api()
                elif api_choice == 5:
                    test_grok_api()
                else:
                    print("請選擇有效的選項")
            except ValueError:
                print("請輸入有效的數字")

            # 詢問是否返回主選單
            if input("\n是否返回主選單? (y/n): ").lower().strip() in ['y', 'yes', '是']:
                main()
            return

        elif choice == 3:
            # 測試所有模型
            test_all_models()

            # 詢問是否返回主選單
            if input("\n是否返回主選單? (y/n): ").lower().strip() in ['y', 'yes', '是']:
                main()
            return

        elif choice == 1:
            # 執行評論分析
            global codebook
            codebook = load_codebook(codebook_path)
            if not codebook:
                print("錯誤: 無法加載代碼簿")
                return

            system_message = build_system_message(codebook)
            reviews = load_reviews(reviews_path)
            if reviews.empty:
                print("錯誤: 無法加載評論資料")
                return

            # 詢問是否測試模型連接
            test_option = input("是否先測試模型連接? (y/n): ").lower().strip()
            if test_option in ['y', 'yes', '是']:
                print("開始測試 API 連接...")
                api_success = test_all_apis()

                if not api_success:
                    print("API 測試失敗，可能影響分析過程")
                    proceed = input("是否仍要繼續? (y/n): ").lower().strip()
                    if proceed not in ['y', 'yes', '是']:
                        return

                print("\n開始測試特定模型連接...")
                if not test_all_models():
                    print("模型測試失敗，請修復問題後再繼續")
                    proceed = input("是否仍要繼續分析? (y/n): ").lower().strip()
                    if proceed not in ['y', 'yes', '是']:
                        return

            # 簡化輸入
            num_reviews = None
            try:
                num_input = input("處理評論數量（按Enter處理全部）: ")
                if num_input.strip():
                    num_reviews = int(num_input)
            except ValueError:
                print("無效輸入，將處理全部評論")
                num_reviews = None

            # 使用全部模型或選擇模型
            select_models_option = input("是否選擇特定模型進行分析? (y/n): ").lower().strip()
            selected_models = None

            if select_models_option in ['y', 'yes', '是']:
                # 獲取所有模型配置
                all_models = get_model_configs()

                # 顯示可用模型
                print("\n可用模型:")
                for i, model in enumerate(all_models, 1):
                    display_name = model.get('display_name', model.get('name'))
                    model_type = model.get('type')
                    print(f"{i}. {display_name} ({model_type})")

                # 選擇模型
                try:
                    model_indices = input("請輸入要使用的模型編號（用逗號分隔，如 1,3,5）: ")
                    if model_indices.strip():
                        indices = [int(idx.strip()) - 1 for idx in model_indices.split(',')]
                        selected_models = [all_models[idx]['name'] for idx in indices if 0 <= idx < len(all_models)]
                        print(f"已選擇 {len(selected_models)} 個模型")
                except ValueError:
                    print("無效輸入，將使用所有模型")
                    selected_models = None

            # 分析評論
            results = analyze_reviews(
                reviews, output_path, log_path, system_message,
                num_reviews=num_reviews, selected_models=selected_models
            )

            print(f"分析完成，結果已保存至 {output_path}")
        else:
            print("請選擇有效的選項")

    except ValueError:
        print("請輸入有效的數字")

    # 詢問是否重新執行主選單
    if input("\n是否返回主選單? (y/n): ").lower().strip() in ['y', 'yes', '是']:
        main()
    else:
        print("程式已退出")
        return

print("✅ 主函數已定義")

✅ 主函數已定義


In [14]:
# Cell 11: 執行主程序

# 全局變量 codebook，用於在不同函數間共享
codebook = []

# 執行主程序
if __name__ == "__main__":
    print("🚀 評論多維度分析工具啟動中...")

    # 檢查檔案路徑
    if check_file_paths():
        print("✅ 檔案檢查通過")
        main()
    else:
        print("❌ 檔案檢查失敗，請檢查必要檔案是否存在")

🚀 評論多維度分析工具啟動中...
已創建 分析結果: multidimensional_analysis_results.json
已創建 日誌檔案: analysis_log.txt
已創建 查詢日誌檔案: query_log.txt
✅ 檔案檢查通過

=== 評論多維度分析工具 ===
1. 進行評論分析
2. 測試 API 連接
3. 測試所有 LLM 模型
4. 人工標註相關功能
5. 互動模型測試
0. 退出程式


請選擇操作:  1


評論資料讀取成功！總筆數: 4952


是否先測試模型連接? (y/n):  n
處理評論數量（按Enter處理全部）:  
是否選擇特定模型進行分析? (y/n):  y



可用模型:
1. DeepSeek R1 1.5B (ollama)
2. DeepSeek R1 8B (ollama)
3. DeepSeek R1 14B (ollama)
4. DeepSeek R1 32B (ollama)
5. Gemma 3 1B (ollama)
6. Gemma 3 4B (ollama)
7. Gemma 3 27B (ollama)
8. Phi-4 14B (ollama)
9. Phi-4 Mini 3.8B (ollama)
10. Qwen 3 0.6B (ollama)
11. Qwen 3 1.7B (ollama)
12. Qwen 3 4B (ollama)
13. Qwen 3 8B (ollama)
14. Qwen 3 14B (ollama)
15. QWQ 32B (ollama)
16. granite3.3:2b (ollama)
17. granite3.3:8b (ollama)
18. Mistral 7B (ollama)
19. Mistral Small 3.1 24B (ollama)
20. Llama 3.2 1B (ollama)
21. Llama 3.2 3B (ollama)
22. gpt-oss:20b (ollama)
23. llama3-Taide (ollama)
24. Breeze2 (ollama)


...


開始使用模型: Mistral Small 3.1 24B (ollama)...


100%|██████████| 4952/4952 [21:08:07<00:00, 15.37s/it]   


分析完成，結果已保存至 multidimensional_analysis_results.json



是否返回主選單? (y/n):  n


程式已退出
